In [2]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from PIL import Image

In [3]:
# Parameters
NUM_FILES = 11000
NUM_CLASSES = 6 
IMG_SIZE = (100, 100)  
BATCH_SIZE = 8  
EPOCHS = 1000
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [4]:
class SpectrogramDataset(Dataset):
    def __init__(self, spectrograms, labels, transform=None):
        self.spectrograms = spectrograms
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.spectrograms)

    def __getitem__(self, idx):
        spectrogram = self.spectrograms[idx]
        label = self.labels[idx]
        if self.transform:
            spectrogram = Image.fromarray(spectrogram)
            if spectrogram.mode != 'RGB':
                spectrogram = spectrogram.convert('RGB')
            spectrogram = self.transform(spectrogram)
        return spectrogram, label

def read_data(data_folder, num_files=None):
    train_spec_folder = os.path.join(data_folder, 'train_spectrograms')
    test_spec_folder = os.path.join(data_folder, 'test_spectrograms')

    def read_npy_folder(folder_path, n_files=None):
        arrays = []
        files_to_read = os.listdir(folder_path)[:n_files] if n_files else os.listdir(folder_path)
        for file in files_to_read:
            if file.endswith('.npy'):
                file_path = os.path.join(folder_path, file)
                array = np.load(file_path)
                arrays.append(array)
        print(f"Read {len(arrays)} files from {folder_path}.")
        return arrays

    train_spec = read_npy_folder(train_spec_folder, num_files)
    test_spec = read_npy_folder(test_spec_folder)

    train_labels = pd.read_csv(os.path.join(data_folder, 'train.csv'), nrows=num_files)
    test_labels = pd.read_csv(os.path.join(data_folder, 'test.csv'))

    return train_spec, test_spec, train_labels, test_labels

# Define your model architecture
class CNNModel(nn.Module):
    def __init__(self, num_classes):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(20000, 128)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        # print("After conv1:", x.size())
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        # print("After conv2:", x.size())
        x = self.relu2(x)
        x = self.pool2(x)
        x = self.flatten(x)
        # print("After flatten:", x.size())
        x = self.fc1(x)
        # print("After fc1:", x.size())
        x = self.relu3(x)
        x = self.fc2(x)
        return x

In [5]:
# Read data
train_spec, test_spec, train_labels, test_labels = read_data('data/npy_data/npy_data', num_files=NUM_FILES)

Read 11000 files from data/npy_data/npy_data/train_spectrograms.
Read 1 files from data/npy_data/npy_data/test_spectrograms.


In [6]:
len(train_spec),train_labels.shape

(11000, (11000, 15))

In [7]:
Xt, Xv, yt, yv = train_test_split(
    train_spec, train_labels,
    test_size=0.2,
    random_state=42,
    shuffle=True
)


In [8]:
y_train = yt.iloc[:, 9:]
y_val = yv.iloc[:, 9:]
pd.DataFrame(y_train)

,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
10735,0,1,0,0,0,14
5937,0,0,3,0,1,8
7642,0,11,0,0,0,6
3328,3,0,0,0,0,0
8681,0,0,0,4,0,2
...,...,...,...,...,...,...
5734,0,0,0,0,1,0
5191,3,0,0,0,0,0
5390,0,0,0,3,0,0
860,1,0,9,0,0,3


In [9]:
y_train = y_train.apply(pd.to_numeric, errors='coerce')
y_train.fillna(0, inplace=True)
y_train_normalized = y_train.div(y_train.sum(axis=1), axis=0)
weights = y_train.sum(axis=1)  # Calculate weights based on number of voters per row
y_train_normalized = y_train_normalized.mul(weights, axis=0)
y_train_normalized = y_train_normalized.div(y_train_normalized.sum(axis=1), axis=0)
y_train = torch.tensor(y_train_normalized.values, dtype=torch.float32)

In [10]:
y_val = y_val.apply(pd.to_numeric, errors='coerce')
y_val.fillna(0, inplace=True)
y_val_normalized = y_val.div(y_val.sum(axis=1), axis=0)
weights = y_val.sum(axis=1)
y_val_normalized = y_val_normalized.mul(weights, axis=0)
y_val_normalized = y_val_normalized.div(y_val_normalized.sum(axis=1), axis=0)
y_val = torch.tensor(y_val_normalized.values, dtype=torch.float32)

In [11]:
np.allclose(y_train_normalized.sum(axis=1), 1),np.allclose(y_val_normalized.sum(axis=1), 1)

(True, True)

In [12]:
model = CNNModel(num_classes=NUM_CLASSES)
model.to(device)
transform = transforms.Compose([
    transforms.Resize(IMG_SIZE),  
    transforms.ToTensor(),       
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]) 
])

train_dataset = SpectrogramDataset(Xt, y_train, transform=transform)
val_dataset = SpectrogramDataset(Xv, y_val, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

criterion = nn.KLDivLoss(reduction='batchmean')
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [13]:
len(train_dataset),len(val_dataset)

(8800, 2200)

In [14]:
for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to the selected device
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(F.log_softmax(outputs, dim=1), labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    
    epoch_loss = running_loss / len(train_loader.dataset)
    if epoch % 10 == 0:
        print(f"Epoch {epoch+1}/{EPOCHS}, Training Loss: {epoch_loss:.4f}")

    # Validation
    model.eval()
    validation_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)  # Move data to the selected device
            outputs = model(inputs)
            loss = criterion(F.log_softmax(outputs, dim=1), labels)
            validation_loss += loss.item() * inputs.size(0)

    epoch_val_loss = validation_loss / len(val_loader.dataset)
    if epoch % 10 == 0:
        print(f"Epoch {epoch+1}/{EPOCHS}, Validation Loss: {epoch_val_loss:.4f}")

Epoch 1/1000, Training Loss: 1.3885
Epoch 1/1000, Validation Loss: 1.3838
Epoch 11/1000, Training Loss: 0.7104
Epoch 11/1000, Validation Loss: 2.1091
Epoch 21/1000, Training Loss: 0.4129
Epoch 21/1000, Validation Loss: 2.3810
Epoch 31/1000, Training Loss: 0.3117
Epoch 31/1000, Validation Loss: 2.7517
Epoch 41/1000, Training Loss: 0.2559
Epoch 41/1000, Validation Loss: 3.0540


KeyboardInterrupt: 